In [16]:
import ollama

import faiss
import pickle
import os
from sentence_transformers import SentenceTransformer
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM

import torch


RAG Pipeline Testing

In [17]:
import pickle
import faiss
import torch
from transformers import AutoTokenizer, AutoModel

def load_retriever(
    index_path: str,
    chunks_path: str
):
    # Initialize device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Load SentenceTransformer model (E5-large)
    model = SentenceTransformer("intfloat/e5-large", device=device)
    
    # Configure for Urdu text
    model.max_seq_length = 512  # Set based on your earlier analysis
    model.tokenizer.do_lower_case = False  # Preserve Urdu characters
    
    # Load FAISS index
    index = faiss.read_index(index_path)
    
    # Load stored chunks
    with open(chunks_path, "rb") as f:
        chunks_list = pickle.load(f)
    
    return model, index, chunks_list, device


In [18]:
model, index, chunks_list, device = load_retriever(
    index_path="../../vector_db/paragraphs/5884_paras/5884_paras_faiss_index.index",
    chunks_path="../../data_storage/Paragraph_chunks/5884_paragraphs/5884_chunks.pkl"
)

In [19]:

import torch.nn.functional as F




def retrieve_documents(query,k=3):
   
    query_embedding = model.encode(
        [query],
        convert_to_tensor=False,  # Return numpy array for FAISS
        normalize_embeddings=True,
        show_progress_bar=False
    )
    query_embedding = model.encode(
        [query],
        convert_to_tensor=False,
        normalize_embeddings=True,
        show_progress_bar=False
    )
    
    # Search FAISS index
    _, indices = index.search(query_embedding, k)  # Dummy variable _ for distances
    
    # Return only the chunks
    return [chunks_list[i] for i in indices[0]]
    



def generate_using_llama3(context, query):
    prompt = f"""You are a helpful assistant. You will be given a context and a question, both written in Urdu.
Your task is to answer the question using only the information from the context.
Your response must:
- Be written entirely in Urdu
- Not include any English words
- Not include translations
- Be clear and concise

Context:
{context}

Question:
{query}

Answer (in Urdu only):"""

    try:
        response = ollama.chat(
            model='llama3:8b',
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response['message']['content'].strip()
    except Exception as e:
        print("Error during generation:", e)
        return "جواب پیدا کرنے میں خرابی ہوئی۔"


In [20]:
def rag_pipeline(query: str, k=3) -> str:
    retrieved_chunks = retrieve_documents(query,k=k)
    print("retrieved_chunks: ", retrieved_chunks)
    answer = generate_using_llama3(query, retrieved_chunks)
    return answer

In [21]:
question = "سیری بی 2017 - 2017 (اسپانسرشپ کی وجوہات کی بناء پر سیری بی کونٹٹ) لیگ کا 86 واں سیزن ہے ، جس میں مجموعی طور پر 22 ٹیمیں مقابلہ کر رہی ہیں: 15 2016 - 17 سیزن سے واپس آرہی ہیں ، 2016 - 17 سیری بی ، جو اسپانسرشپ کی وجوہات کی بناء پر سیری بی کونٹٹٹ کے نام سے جانا جاتا ہے ، اور اس کے قیام کے بعد سے 85 واں سیزن تھا۔"
ans=rag_pipeline(question)
ans

retrieved_chunks:  ['سیری بی (اسپانسرشپ وجوہات کی بناء پر سیری بی) کا مقابلہ کرنے والی لیگ کی 86 ویں سیزن ہے۔ اس میں مجموعی طور پر 22 ٹیمیں حصہ لے رہی ہیں: 15 2016 سے واپس آ رہی ہیں ، 4 لیگا پرو سے ترقی یافتہ ہیں ، اور 3 سیری اے سے نیچے گر چکے ہیں۔', '۔ اس کا بڑا بھائی ، اساک برینسٹروم ، بھی ایک ایس ایچ ایل کھلاڑی ہے ، فی الحال ایچ ایچ ایل کے ساتھ۔ وہ 2017 کے این ایچ ایل داخلہ ڈرافٹ میں گولڈن نائٹس کے ذریعہ مجموعی طور پر 15 ویں نمبر پر منتخب کیا گیا تھا۔', '۔ اس سیریز کو سارہ ڈن نے تخلیق کیا اور لکھا اور ہارون کیپلن ، کینی شوارٹز ، ریک وینر ، اور روبن فلیشر کے ساتھ شریک ایگزیکٹو پروڈیوسر ، کیپٹل انٹرٹینمنٹ اسٹوڈیوز کے ساتھ مشترکہ پروڈکشن۔ اس سیریز کا ایک پیش نظارہ 17 مئی ، 2016 کو جاری کیا گیا تھا']


'سیری بی (اسپانسرشپ وجوہات کی بناء پر سیری بی) کے مقابلہ کرنے والی لیگ کی سیزن 86 ہے ، جس میں مجموعی طور پر 22 ٹیمیں حصہ لے رہی ہیں: 15 2016 سے واپس آ رہی ہیں ، 4 لیگا پرو سے ترقی یافتہ ہیں ، اور 3 سیری اے سے نیچے گر چکے ہیں۔'

RAG Pipeline testing on Dataset


In [23]:
import os
print(os.listdir("../../results/pipeline results"))  # Show top-level folders

['100paras_100qna', '5884paras_598qna']


In [24]:
import pandas as pd



# Load only the required columns
df = pd.read_csv('../../../Dataset_code_csvs/hotpotQA/hotpotQA_dataset_versions/5884paras_598queries/Urdu/598_QnAs_translated.csv', usecols=[
    'level', 'translated_question', 'translated_answer', 'translated_retrieved_sentences'
])

# Rename the column
df.rename(columns={'translated_retrieved_sentences': 'translated_context'}, inplace=True)

# Optional: View the result
print(df.head())

  level                                translated_question translated_answer  \
0  easy  سیری بی 2017 - 2017 (اسپانسرشپ کی وجوہات کی بن...         1929ء میں   
1  easy  "آکسفورڈ کالج کے ایک ساتھی ایلک نیلر ڈکن نے کہ...       کرپٹوولوجسٹ   
2  easy  "جراسک پارک کے اداکار ڈیوڈ ہنری ہوانگ نے ""دی ...        بی ڈی وانگ   
3  easy  کون سا کردار ، ڈین کاسٹیلینیٹا کی آواز ، سمپسن...        دادا سمپسن   
4  easy     کون تھا ایک حصہ S#arp، لی Ji-hye یا کرٹس رائٹ؟          لی جی ہے   

                                  translated_context  
0  لیگ میں مجموعی طور پر 22 ٹیمیں حصہ لے رہی ہیں:...  
1  ایلکلر ڈکن (انگریزی: Alec Dakin) (۳ اپریل ۱۹۱۲...  
2  "جیرز زکس نے اس فلم کی ہدایت کاری کی تھی جس می...  
3  "سیمپسنز کے 22 ویں سیزن کی دوسری قسط "" لون اے...  
4  لی جی ہیو (پیدائش 11 جنوری ، 1980) ایک جنوبی ک...  


In [25]:
import os
import time
from datetime import timedelta

# Initialize empty columns
df['retrieved_context'] = ""
df['final_answer'] = ""
df['retriever_time'] = 0.0
df['generator_time'] = 0.0
df['total_time'] = 0.0

# Relative path to output directory
output_dir = "../../results/pipeline results/5884paras_598qna"
os.makedirs(output_dir, exist_ok=True)
output_csv = os.path.join(output_dir, "simple_rag_qna_results.csv")

# Timing variables
total_start_time = time.time()
batch_start_time = time.time()
processed_count = 0

print(f"Starting processing of {len(df)} records at {time.strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

for i, row in df.iterrows():
    record_start_time = time.time()
    query = row['translated_question']
    
    # Print current record being processed
    print(f"\nProcessing record {i+1}...")  # Show first 50 chars of query
    
    # Retrieve documents
    retriever_start = time.time()
    retrieved_chunks = retrieve_documents(query, k=3)
    retriever_time = time.time() - retriever_start
    
    # Generate answer
    generator_start = time.time()
    final_answer = generate_using_llama3(query, "\n".join(retrieved_chunks))
    generator_time = time.time() - generator_start
    
    # Update dataframe
    df.at[i, 'retrieved_context'] = "\n".join(retrieved_chunks)
    df.at[i, 'final_answer'] = final_answer
    df.at[i, 'retriever_time'] = retriever_time
    df.at[i, 'generator_time'] = generator_time
    df.at[i, 'total_time'] = time.time() - record_start_time
    
    # Print record processing time
    print(f"Completed record {i+1} in {df.at[i, 'total_time']:.2f}s "
          f"(Retriever: {retriever_time:.2f}s, Generator: {generator_time:.2f}s)")
    
    # Save progress every 100 records
    if (i + 1) % 100 == 0 or (i + 1) == len(df):
        batch_end_time = time.time()
        batch_duration = batch_end_time - batch_start_time
        processed_count = min(100, (i+1) - (i//100)*100)  # Handle partial batches
        
        print("\n" + "="*60)
        print(f"BATCH SUMMARY: Records {(i//100)*100 + 1}-{i+1}")
        print(f"Batch processing time: {timedelta(seconds=batch_duration)}")
        print(f"Average time per record: {batch_duration/processed_count:.2f}s")
        print(f"Current timestamp: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        # Save batch
        df.iloc[max(0, i-99):i+1].to_csv(
            output_csv,
            mode='a',
            header=not os.path.exists(output_csv),
            index=False,
            encoding="utf-8-sig"
        )
        print(f"Saved batch to: {os.path.abspath(output_csv)}")
        print("="*60 + "\n")
        
        batch_start_time = time.time()

# Final statistics
total_duration = time.time() - total_start_time
print("\n" + "="*80)
print(f"PROCESSING COMPLETED: {len(df)} records")
print(f"Total processing time: {timedelta(seconds=total_duration)}")
print(f"Average time per record: {total_duration/len(df):.2f}s")
print(f"Total retriever time: {df['retriever_time'].sum():.2f}s")
print(f"Total generator time: {df['generator_time'].sum():.2f}s")
print("="*80)

Starting processing of 598 records at 2025-05-19 02:35:01

Processing record 1...
Completed record 1 in 27.44s (Retriever: 0.43s, Generator: 27.01s)

Processing record 2...
Completed record 2 in 33.82s (Retriever: 0.26s, Generator: 33.56s)

Processing record 3...
Completed record 3 in 5.93s (Retriever: 0.23s, Generator: 5.70s)

Processing record 4...
Completed record 4 in 25.21s (Retriever: 0.27s, Generator: 24.95s)

Processing record 5...
Completed record 5 in 4.13s (Retriever: 0.23s, Generator: 3.90s)

Processing record 6...
Completed record 6 in 46.88s (Retriever: 0.32s, Generator: 46.56s)

Processing record 7...
Completed record 7 in 14.01s (Retriever: 0.35s, Generator: 13.66s)

Processing record 8...
Completed record 8 in 33.24s (Retriever: 0.15s, Generator: 33.09s)

Processing record 9...
Completed record 9 in 12.16s (Retriever: 0.15s, Generator: 12.02s)

Processing record 10...
Completed record 10 in 8.41s (Retriever: 0.22s, Generator: 8.19s)

Processing record 11...
Completed r

In [ ]:
import ollama

def generate_using_alif(query, context, alif_model='hf.co/large-traversaal/Alif-1.0-8B-Instruct:f16'):


    prompt = f"""آپ کو ایک سوال اور اس سے متعلق ایک سیاق و سباق دیا گیا ہے۔ براہ کرم سیاق و سباق کا بغور مطالعہ کریں اور اسی کی بنیاد پر درست، مختصر اور جامع جواب دیں۔

### سوال:
{query}

### سیاق و سباق:
{context}

### جواب:
"""

    response = ollama.chat(
        model=alif_model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        stream=False
    )

    return response['message']['content']
